In [9]:
import numpy as np
import pandas as pd
import csv
import os
import glob
import tensorflow as tf

from PIL import Image

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.models import Model

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from keras.optimizers import Adam
from keras import backend as K

In [10]:
physical_devices = tf.config.list_physical_devices('GPU')
for i, device in enumerate(physical_devices):
    print(f"GPU {i}: {device}")


GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [11]:
path = 'car_make_images/'
training_path = path + 'train'
testing_path = path + 'test'
validation_path = path + 'val'

In [12]:
training_data_generator = ImageDataGenerator(rescale = 1./255,
                              rotation_range = 359,
                              shear_range = 0.2,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              zoom_range = 0.2,
                              horizontal_flip = True,
                              vertical_flip = True,
                              preprocessing_function = None)

validation_data_generator = ImageDataGenerator(rescale = 1./255)
test_data_generator = ImageDataGenerator(rescale = 1./255)

In [13]:
size = 512
batch_size = 32  
num_classes = 39

training_generator = training_data_generator.flow_from_directory(training_path,
                                                                 target_size = (size, size),
                                                                 batch_size = 30,
                                                                 class_mode = "categorical",
                                                                 color_mode = 'grayscale',
                                                                 )

validation_generator = validation_data_generator.flow_from_directory(validation_path,
                                                                     target_size = (size, size),
                                                                     batch_size = 1,
                                                                     class_mode = "categorical",
                                                                     color_mode = 'grayscale',
                                                                     )

test_generator = test_data_generator.flow_from_directory(testing_path,
                                                         target_size = (size, size),
                                                         batch_size = 1,
                                                         class_mode = "categorical",
                                                         color_mode = 'grayscale',
                                                         shuffle=False
                                                         )

Found 11573 images belonging to 39 classes.
Found 2813 images belonging to 39 classes.
Found 2871 images belonging to 39 classes.


In [14]:
model = Sequential()

model.add(Input(shape=(size, size, 1)))

# First Conv Block
model.add(Conv2D(filters=16, kernel_size=7, padding='same', kernel_initializer='he_normal'))
# model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling2D(pool_size=2))

# Second Conv Block
model.add(Conv2D(filters=32, kernel_size=5, padding='same', kernel_initializer='he_normal'))
# model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling2D(pool_size=2))

# Third Conv Block
model.add(Conv2D(filters=64, kernel_size=3, padding='same', kernel_initializer='he_normal'))
# model.add(BatchNormalization())
model.add(tf.keras.layers.ReLU())
model.add(MaxPooling2D(pool_size=2))

# Flatten and Fully Connected Layers
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.15))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.15))
model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(num_classes, activation='softmax', kernel_initializer='he_normal'))

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=0.001,
            decay_steps=10000,
            decay_rate=0.9)
opt = Adam(learning_rate=lr_schedule)

In [15]:
def train_and_save(model, train_data, val_data, epochs, save_interval, model_save_path, history_save_path, custom_metrics=None, custom_optimizer=None):
    """
    Train a TensorFlow model and save it along with its history.
    """
    os.makedirs(model_save_path, exist_ok=True)

    if custom_optimizer:
        optimizer = custom_optimizer
    else:
        optimizer = 'adam'

    if custom_metrics:
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['accuracy'] + custom_metrics)
    else:
        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

    # Initialize variables
    initial_epoch = 0
    temp_history_data = []

    # Check if history file exists, if not create it
    if not os.path.exists(history_save_path):
        with open(history_save_path, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            columns = ['Epoch', 'Loss', 'Accuracy', 'Val_Loss', 'Val_Accuracy']
            if custom_metrics:
                for metric in custom_metrics:
                    metric_name = metric.__name__
                    columns.append(metric_name)
                    columns.append("Val_" + metric_name)
            csv_writer.writerow(columns)
    else:
        with open(history_save_path, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            last_row = None
            for row in csv_reader:
                last_row = row
            if last_row:
                initial_epoch = int(last_row[0])

    latest_model_file = max(glob.glob(f"{model_save_path}/model_e*.h5"), default=None, key=os.path.getctime)
    if latest_model_file is not None:
        print(f"Resuming from {latest_model_file}")
        model = tf.keras.models.load_model(latest_model_file, custom_objects={metric.__name__: metric for metric in custom_metrics})

    for epoch in range(initial_epoch + 1, epochs + initial_epoch + 1):
        print(f"Epoch {epoch}/{epochs + initial_epoch}")

        history = model.fit(train_data, validation_data=val_data)
        history_data = [epoch] + [history.history[key][0] for key in history.history]
        temp_history_data.append(history_data)

        if epoch % save_interval == 0 or epoch == epochs + initial_epoch:
            model_file_path = os.path.join(model_save_path, f"model_e{epoch}.h5")
            model.save(model_file_path)

            # Append to CSV at checkpoints
            with open(history_save_path, 'a', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)
                for row in temp_history_data:
                    csv_writer.writerow(row)

            # Clear temporary history data
            temp_history_data.clear()

            print(f"Saved model and history at epoch {epoch}")


In [16]:
# Train the model and save it

train_and_save(
    model, 
    training_generator, 
    validation_generator, 
    epochs=100, 
    save_interval=10, 
    model_save_path="models", 
    history_save_path="history.csv", 
    custom_metrics=[],
    custom_optimizer=opt
)


Epoch 1/100
386/386 [==============================] - 485s 1s/step - loss: 4.2745 - accuracy: 0.0366 - val_loss: 3.6248 - val_accuracy: 0.0430
Epoch 2/100
386/386 [==============================] - 330s 854ms/step - loss: 3.6278 - accuracy: 0.0419 - val_loss: 3.6192 - val_accuracy: 0.0459
Epoch 3/100
386/386 [==============================] - 331s 857ms/step - loss: 3.6195 - accuracy: 0.0445 - val_loss: 3.6105 - val_accuracy: 0.0483
Epoch 4/100
386/386 [==============================] - 331s 857ms/step - loss: 3.6099 - accuracy: 0.0449 - val_loss: 3.6130 - val_accuracy: 0.0501
Epoch 5/100
386/386 [==============================] - 336s 870ms/step - loss: 3.6028 - accuracy: 0.0479 - val_loss: 3.5938 - val_accuracy: 0.0526
Epoch 6/100
386/386 [==============================] - 333s 863ms/step - loss: 3.5962 - accuracy: 0.0526 - val_loss: 3.5865 - val_accuracy: 0.0540
Epoch 7/100
386/386 [==============================] - 334s 864ms/step - loss: 3.5884 - accuracy: 0.0518 - val_loss: 3.57

KeyboardInterrupt: 

In [ ]:
saKOJHSDJHKahskjA
# train_and_save(
#     model, 
#     training_generator, 
#     validation_generator, 
#     epochs=150, 
#     save_interval=10, 
#     model_save_path="models", 
#     history_save_path="history.csv", 
#     custom_metrics=[],
#     custom_optimizer=opt
# )

Resuming from models\model_e500.h5
Epoch 501/650
386/386 [==============================] - 273s 704ms/step - loss: 1.2068 - accuracy: 0.6456 - val_loss: 4.3441 - val_accuracy: 0.3249
Epoch 502/650
386/386 [==============================] - 158s 410ms/step - loss: 1.2154 - accuracy: 0.6467 - val_loss: 4.3033 - val_accuracy: 0.3374
Epoch 503/650
386/386 [==============================] - 160s 413ms/step - loss: 1.2074 - accuracy: 0.6449 - val_loss: 4.2728 - val_accuracy: 0.3324
Epoch 504/650
386/386 [==============================] - 172s 445ms/step - loss: 1.1901 - accuracy: 0.6542 - val_loss: 4.6231 - val_accuracy: 0.3146
Epoch 505/650
386/386 [==============================] - 171s 441ms/step - loss: 1.1991 - accuracy: 0.6529 - val_loss: 4.7306 - val_accuracy: 0.3171
Epoch 506/650
386/386 [==============================] - 179s 465ms/step - loss: 1.1803 - accuracy: 0.6522 - val_loss: 4.3119 - val_accuracy: 0.3370
Epoch 507/650
386/386 [==============================] - 180s 466ms/ste

In [ ]:
import pandas as pd
import plotly.express as px

model = tf.keras.models.load_model('models/model_e650.h5')

model.evaluate(test_generator)

2871/2871 [==============================] - 119s 41ms/step - loss: 4.8088 - accuracy: 0.3285


[4.808804035186768, 0.32845696806907654]

In [ ]:
predictions = model.predict(test_generator)

predicted_labels = np.argmax(predictions, axis = 1)

2871/2871 [==============================] - 33s 11ms/step


In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(test_generator.classes, predicted_labels)

In [ ]:
normalized_confusion = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis]

# Create a DataFrame for the heatmap
df = pd.DataFrame(normalized_confusion, index=list(range(39)), columns=list(range(39)))
df.to_csv('confusion_matrix_data.csv', index=False)

# Create the heatmap using Plotly Express
fig = px.imshow(df,
                x=list(range(39)),
                y=list(range(39)),
                color_continuous_scale='Viridis',  # Choose your desired color scale
                labels=dict(x='Predicted Labels', y='True Labels', color='Normalized Confusion')
                )

# Customize the layout
fig.update_layout(title='Confusion Matrix Heatmap')

# Show the plot
fig.show()